In [23]:
import torch
import torchvision
from torch import nn
from torch.nn import Linear
from torch.nn import Conv2d, MaxPool2d, Flatten, Sequential
from torch.nn import L1Loss, MSELoss, CrossEntropyLoss
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os
import shutil


### subdirectory

In [30]:
# subdirectory
now = datetime.now()
logdir = "tb_logs/" + now.strftime("%Y%m%d-%H%M%S") + "/"

# Check if the directoryexists
if os.path.exists(logdir):
    # Delete the directory and all its contents
    shutil.rmtree(logdir)

# recreate the directory if plan to continue logging
os.makedirs(logdir, exist_ok=True)


### 創建資料

In [14]:
dataset = torchvision.datasets.CIFAR10("../data", train=False, transform=torchvision.transforms.ToTensor(), download=True)
dataloader = DataLoader(dataset, batch_size=1)

Files already downloaded and verified


In [21]:
class nn_model_vgg16(nn.Module):
    def __init__(self):
        super(nn_model_vgg16, self).__init__()
           
        self.model = Sequential(
            Conv2d(3,32,5, padding=2),
            MaxPool2d(2),
            Conv2d(32,32,5, padding=2),
            MaxPool2d(2),
            Conv2d(32,64,5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x


In [22]:
loss = nn.CrossEntropyLoss()
nn_model_vgg16 = nn_model_vgg16()

optim = torch.optim.SGD(nn_model_vgg16.parameters(), lr=0.01)

# 學習調整器
scheduler = lr_scheduler.StepLR(optim, step_size=5, gamma=0.5)

for epoch in range(1):
    running_loss = 0.0
    for data in dataloader:
        imgs, targets = data
        outputs = nn_model_vgg16(imgs)
        output_loss = loss(outputs, targets)
        optim.zero_grad()
        output_loss.backward()
        scheduler.step()
        running_loss = running_loss + output_loss
    print(running_loss)



tensor(18706.3672, grad_fn=<AddBackward0>)
